In [1]:
import pypsa
import pandas as pd
import geopandas as gpd

In [ ]:
nodes = gpd.read_file('../data/spatial/network/nodes.shp')
edges = gpd.read_file('../data/spatial/network/edges.shp')
loads = pd.read_csv('../data/csv/generated_nodal_flows.csv')

# Drop duplicates
initial_count = nodes.shape[0]
nodes.drop_duplicates(subset='id', inplace=True)
final_count = nodes.shape[0]
print(f"Dropped {initial_count - final_count} duplicate nodes.")

# Drop duplicates
initial_count = edges.shape[0]
edges.drop_duplicates(subset='id', inplace=True)
final_count = edges.shape[0]
print(f"Dropped {initial_count - final_count} duplicate edges.")

# Ensure all junction nodes are named substations
nodes.loc[nodes['asset_type'] == 'junction', 'subtype'] = 'substation'

# remove any edges that have a from_id or to_id that is not in the nodes
nodes_ids = nodes['id'].tolist()
edges = edges[edges['from_id'].isin(nodes_ids) & edges['to_id'].isin(nodes_ids)].reset_index(drop=True)

# remove any loads that have a node_id that is not in the nodes
loads = loads[ [i for i in loads.columns if i in nodes.id.unique()] ]

# re-map generators to nearest substation
# TODO!!!!

In [ ]:
from pyproj import Transformer
from tqdm import tqdm

network = pypsa.Network()

junctions = nodes.query(" asset_type == 'junction' ")

# Add substations as buses
transformer = Transformer.from_crs("EPSG:3448", "EPSG:4326")
for _, n in tqdm(junctions.iterrows(), total=junctions.shape[0], desc="Adding junctions"): 
    lon, lat = transformer.transform(n.geometry.x, n.geometry.y)

    # Add generators
    network.add(
        "Bus",
        n.id,
        x = lat,
        y = lon,
    )

# Add lines
edge_set = edges.loc[ (edges.from_type == 'junction') & (edges.to_type == 'junction') ]
for _, e in tqdm(edge_set.iterrows(), total=edge_set.shape[0], desc="Adding edges"):
    network.add(
        "Link",
        e.id,
        bus0=e.from_id,
        bus1=e.to_id,
        p_nom=1e12,
    )

# Add generators
generators = nodes.query(" asset_type == 'source' ")
for _, n in tqdm(generators.iterrows(), total=generators.shape[0], desc="Adding generators"):
    network.add(
        "Generator",
        n.id,
        bus=edges.loc[edges.from_id == n.id, 'to_id'].iloc[0],
        p_nom=n.capacity,
        p_nom_extendable=False,
        marginal_cost=1,
    )

# Add loads
for col in tqdm(loads.columns, desc="Adding loads"):
    try:
        network.add(
            "Load",
            col,
            bus=edges.loc[edges.to_id == col, 'from_id'].iloc[0],
            p_set=loads[col].to_numpy(),
        )
    except:
        print(f"Failed to add load {col}")
        continue

# export
network.export_to_netcdf('../data/networks/elec.nc')

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs  # Import Cartopy's coordinate reference systems

# Create a GeoAxesSubplot
fig, ax = plt.subplots(1, 1, figsize=(15, 10), subplot_kw={'projection': ccrs.Mercator()})

# Plot the network on the GeoAxesSubplot
network.plot(
    geomap=True, 
    margin=0.1, 
    bus_sizes=0.00002, 
    ax=ax,
    bus_colors="#00C0B0",
    bus_alpha=0.2,
    line_colors="#FFB405",
    link_colors='#FFB405',
    link_widths=0.5,
    color_geomap = {
        "ocean": "#1F283D",
        "land": "#22293d",
        "border": "white",
        "coastline": "white",
    }
)

plt.legend()
plt.show()

In [ ]:
log_text = """
Adding edges:   5%|▌         | 3110/61440 [00:22<07:09, 135.95it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_4386'], dtype='object')
Adding edges:   5%|▌         | 3124/61440 [00:22<07:12, 134.73it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_4387'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_4401'], dtype='object')
Adding edges:   6%|▌         | 3768/61440 [00:27<07:10, 134.00it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_5328'], dtype='object')
Adding edges:   7%|▋         | 4169/61440 [00:30<07:30, 127.20it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_5882'], dtype='object')
Adding edges:   7%|▋         | 4221/61440 [00:30<07:34, 125.77it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_5952'], dtype='object')
Adding edges:   7%|▋         | 4234/61440 [00:30<07:39, 124.49it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_5963'], dtype='object')
Adding edges:   7%|▋         | 4286/61440 [00:31<07:52, 121.03it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_6030'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_6031'], dtype='object')
Adding edges:   7%|▋         | 4547/61440 [00:33<07:33, 125.36it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_6455'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_6456'], dtype='object')
Adding edges:   7%|▋         | 4599/61440 [00:33<07:32, 125.61it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_6541'], dtype='object')
Adding edges:   8%|▊         | 4664/61440 [00:34<07:27, 126.89it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_6619'], dtype='object')
Adding edges:  15%|█▌        | 9237/61440 [01:13<07:31, 115.51it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_13299'], dtype='object')
Adding edges:  15%|█▌        | 9357/61440 [01:14<07:39, 113.22it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_13493'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_13498'], dtype='object')
Adding edges:  15%|█▌        | 9405/61440 [01:14<07:42, 112.63it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_13575'], dtype='object')
Adding edges:  15%|█▌        | 9477/61440 [01:15<07:42, 112.37it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_13673'], dtype='object')
Adding edges:  17%|█▋        | 10223/61440 [01:22<08:17, 102.85it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_14752'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_14753'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_14753'], dtype='object')
Adding edges:  18%|█▊        | 10888/61440 [01:27<07:16, 115.92it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_15632'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_15633'], dtype='object')
Adding edges:  21%|██        | 13038/61440 [01:47<07:08, 112.85it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_18577'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_18578'], dtype='object')
Adding edges:  23%|██▎       | 14372/61440 [01:59<07:48, 100.54it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_20639'], dtype='object')
Adding edges:  36%|███▌      | 22052/61440 [03:20<07:04, 92.87it/s] WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_31792'], dtype='object')
Adding edges:  49%|████▊     | 29857/61440 [04:51<06:21, 82.79it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_42955'], dtype='object')
Adding edges:  49%|████▉     | 29956/61440 [04:52<06:22, 82.38it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_43082'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_43082'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_43083'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_43083'], dtype='object')
Adding edges:  50%|████▉     | 30704/61440 [05:01<06:19, 81.04it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_44165'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_44165'], dtype='object')
Adding edges:  55%|█████▌    | 33836/61440 [05:41<06:03, 75.85it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_48559'], dtype='object')
Adding edges:  55%|█████▌    | 33844/61440 [05:41<05:58, 76.98it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_48560'], dtype='object')
Adding edges:  55%|█████▌    | 33852/61440 [05:41<06:05, 75.46it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_48574'], dtype='object')
Adding edges:  56%|█████▌    | 34484/61440 [05:50<05:58, 75.24it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_49501'], dtype='object')
Adding edges:  57%|█████▋    | 34899/61440 [05:55<05:52, 75.26it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50055'], dtype='object')
Adding edges:  57%|█████▋    | 34947/61440 [05:56<05:49, 75.80it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50125'], dtype='object')
Adding edges:  57%|█████▋    | 34955/61440 [05:56<05:56, 74.34it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50136'], dtype='object')
Adding edges:  57%|█████▋    | 35003/61440 [05:57<05:58, 73.77it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50203'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50204'], dtype='object')
Adding edges:  57%|█████▋    | 35268/61440 [06:01<06:14, 69.86it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50628'], dtype='object')
Adding edges:  57%|█████▋    | 35276/61440 [06:01<06:08, 71.10it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50629'], dtype='object')
Adding edges:  57%|█████▋    | 35324/61440 [06:01<05:54, 73.68it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50714'], dtype='object')
Adding edges:  58%|█████▊    | 35383/61440 [06:02<06:35, 65.94it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_50792'], dtype='object')
Adding edges:  65%|██████▌   | 39960/61440 [07:10<05:06, 70.02it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_57472'], dtype='object')
Adding edges:  65%|██████▌   | 40075/61440 [07:11<05:12, 68.40it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_57666'], dtype='object')
Adding edges:  65%|██████▌   | 40082/61440 [07:12<05:43, 62.20it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_57671'], dtype='object')
Adding edges:  65%|██████▌   | 40132/61440 [07:12<05:12, 68.11it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_57748'], dtype='object')
Adding edges:  65%|██████▌   | 40197/61440 [07:13<05:12, 67.89it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_57846'], dtype='object')
Adding edges:  67%|██████▋   | 40947/61440 [07:25<05:12, 65.66it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_58925'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_58926'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_58926'], dtype='object')
Adding edges:  68%|██████▊   | 41610/61440 [07:34<04:55, 67.10it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_59805'], dtype='object')
Adding edges:  68%|██████▊   | 41617/61440 [07:35<05:18, 62.27it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_59806'], dtype='object')
Adding edges:  71%|███████   | 43757/61440 [08:08<04:37, 63.72it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_62750'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_62751'], dtype='object')
Adding edges:  73%|███████▎  | 45093/61440 [08:30<04:41, 58.04it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_64812'], dtype='object')
Adding edges:  86%|████████▌ | 52774/61440 [10:46<02:41, 53.52it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_75965'], dtype='object')
Adding edges:  99%|█████████▊| 60578/61440 [13:19<00:17, 49.04it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_87128'], dtype='object')
Adding edges:  99%|█████████▉| 60678/61440 [13:21<00:15, 49.35it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_87255'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_87255'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_87256'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_87256'], dtype='object')
Adding edges: 100%|█████████▉| 61430/61440 [13:37<00:00, 49.27it/s]WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_88338'], dtype='object')
WARNING:pypsa.io:The following Link have buses which are not defined:
Index(['edge_88338'], dtype='object')
Adding edges: 100%|██████████| 61440/61440 [13:37<00:00, 75.15it/s]
Adding generators:   0%|          | 0/11 [00:00<?, ?it/s]WARNING:pypsa.io:The following Generator have buses which are not defined:
Index(['node_61'], dtype='object')
WARNING:pypsa.io:The following Generator have buses which are not defined:
Index(['node_62'], dtype='object')
"""

import re
edge_ids = re.findall(r"edge_\d+", log_text)
edge_ids

In [ ]:
edges.set_index('id').loc[edge_ids]

In [ ]:
nodes.set_index('id').loc[['node_6002']]